# Kafka Security Event Viewer

This notebook connects to the lab's **Kafka event bus** and lets you observe, filter, and analyze security events in real-time. It is part of the [Event-Driven Certificate Revocation Lab](https://github.com/czinda/cert-revocation-lab), which demonstrates automated certificate lifecycle management across a **triple-PKI infrastructure**.

## What this lab does

The lab runs three independent PKI hierarchies — **RSA-4096**, **ECC P-384**, and **ML-DSA-87** (post-quantum) — each managed by Dogtag PKI. When a security incident is detected (e.g., key compromise, ransomware, lateral movement), the event flows through a real-time pipeline:

```
Mock EDR / SIEM  ──▶  Kafka (security-events topic)  ──▶  Event-Driven Ansible  ──▶  Dogtag Revocation
```

**This notebook** lets you tap into the Kafka leg of that pipeline: view recent events, stream them live, filter by PKI type or severity, and compute statistics.

## Prerequisites

- Kafka must be running (`kafka.cert-lab.local:9092`)
- The `security-events` topic must exist
- Start the lab with `./start-lab.sh --all` (or `--rsa`, `--ecc`, `--pqc` for a single hierarchy)

## Lab Architecture & Event Types

### Triple-PKI Infrastructure

| PKI Type | Algorithm | Network | Ports (Root / Inter / IoT / EST) |
|----------|-----------|---------|----------------------------------|
| **RSA-4096** | SHA-512 with RSA | 172.26.0.0/24 | 8443 / 8444 / 8445 / 8447 |
| **ECC P-384** | ECDSA with SHA-384 | 172.28.0.0/24 | 8463 / 8464 / 8465 / 8466 |
| **ML-DSA-87** | NIST FIPS 204 Level 5 | 172.27.0.0/24 | 8453 / 8454 / 8455 / 8456 |

Each hierarchy has its own Root CA → Intermediate CA → IoT Sub-CA / EST Sub-CA chain.

### 26 Security Event Types (6 Categories)

| Category | Event Types |
|----------|-------------|
| **Original** | `malware_detection`, `credential_theft`, `ransomware`, `c2_communication`, `lateral_movement`, `privilege_escalation`, `suspicious_script` |
| **PKI/Cert** | `key_compromise`, `geo_anomaly`, `compliance_violation`, `mitm_detected`, `rogue_ca` |
| **IoT** | `firmware_integrity`, `device_cloning`, `iot_anomaly`, `protocol_attack` |
| **Identity** | `impossible_travel`, `service_account_abuse`, `mfa_bypass`, `kerberoasting` |
| **Network** | `tls_downgrade`, `ct_log_mismatch`, `ocsp_bypass` |
| **SIEM** | `data_exfiltration`, `unauthorized_access`, `certificate_misuse` |

The EDA rulebook has **87 rules** — each event type has explicit RSA/ECC/PQC routing (no catch-all fallback), plus 4 FreeIPA identity rules and 2 logging rules.

## Configuration

The Jupyter container is connected to the main lab network (`172.20.0.0/16`) and can reach Kafka directly by hostname.

| Environment Variable | Default | Description |
|---------------------|---------|-------------|
| `KAFKA_BOOTSTRAP_SERVERS` | `kafka.cert-lab.local:9092` | Kafka broker address. Set in `podman-compose.yml` for the Jupyter service. |
| `KAFKA_TOPIC` | `security-events` | The topic where EDR/SIEM events are published. Created automatically by the lab. |

These are pre-configured in the container — you should not need to change them.

In [ ]:
import os
import json
from datetime import datetime
from kafka import KafkaConsumer, KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
import pandas as pd
from IPython.display import display, clear_output
import time

In [ ]:
# Configuration
KAFKA_SERVERS = os.getenv('KAFKA_BOOTSTRAP_SERVERS', 'kafka.cert-lab.local:9092')
KAFKA_TOPIC = os.getenv('KAFKA_TOPIC', 'security-events')

print(f"Kafka Servers: {KAFKA_SERVERS}")
print(f"Topic: {KAFKA_TOPIC}")

## Check Kafka Connection

In [ ]:
try:
    admin = KafkaAdminClient(bootstrap_servers=KAFKA_SERVERS)
    topics = admin.list_topics()
    print(f"Connected to Kafka!")
    print(f"Available topics: {topics}")
    admin.close()
except Exception as e:
    print(f"Failed to connect: {e}")

## Event Payload Schema

Each message on the `security-events` Kafka topic is a JSON object. Below are the key fields you will see in the DataFrames throughout this notebook.

| Field | Type | Description | Example |
|-------|------|-------------|---------|
| `event_id` | string | Unique event identifier | `"evt-abc123"` |
| `timestamp` | string | ISO 8601 timestamp | `"2025-01-15T10:30:00Z"` |
| `event_type` | string | One of the 26 event types (see table above) | `"key_compromise"` |
| `severity` | string | `critical`, `high`, `medium`, or `low` | `"critical"` |
| `device_fqdn` | string | Fully qualified hostname of the affected device | `"sensor01.cert-lab.local"` |
| `description` | string | Human-readable description of the incident | `"Private key exposure detected"` |
| `pki_type` | string | Target PKI hierarchy: `rsa`, `ecc`, or `pqc` | `"rsa"` |
| `ca_level` | string | CA that issued the cert: `root`, `intermediate`, `iot`, `est`, `acme` | `"iot"` |
| `certificate_serial` | string | Serial number of the certificate to revoke (hex, `0x` prefix) | `"0x1A2B3C"` |
| `action_required` | string | Recommended remediation action | `"revoke_certificate"` |

**Note:** Not every event carries all fields. For example, `pki_type` and `certificate_serial` are set by the `./lab test` workflow when it issues a certificate before triggering the event. Events from the Mock EDR/SIEM may omit them, in which case EDA defaults to the RSA hierarchy.

## View Recent Events

Read the last N events from the topic.

In [ ]:
def get_recent_events(n=10):
    """Get the most recent N events from Kafka."""
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_SERVERS,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        consumer_timeout_ms=5000,
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )
    
    events = []
    for message in consumer:
        events.append(message.value)
    
    consumer.close()
    return events[-n:] if len(events) > n else events

# Get recent events
events = get_recent_events(20)
print(f"Found {len(events)} events")

In [ ]:
if events:
    # Convert to DataFrame for nice display
    df = pd.DataFrame(events)
    
    # Select key columns if they exist
    columns = ['timestamp', 'event_type', 'device_fqdn', 'severity', 'description', 'certificate_serial']
    available_cols = [c for c in columns if c in df.columns]
    
    display(df[available_cols] if available_cols else df)
else:
    print("No events found. Trigger some events using the Mock EDR!")

## Filter Events by PKI Type

When running the lab with multiple PKI hierarchies (`--all` or `--dual`), you can isolate activity for a specific PKI type. Change `PKI_FILTER` below to `"rsa"`, `"ecc"`, `"pqc"`, or `None` to see all events.

In [ ]:
# Set to "rsa", "ecc", "pqc", or None for all events
PKI_FILTER = None

if events:
    df_all = pd.DataFrame(events)

    if PKI_FILTER and 'pki_type' in df_all.columns:
        df_filtered = df_all[df_all['pki_type'] == PKI_FILTER]
        print(f"Showing {len(df_filtered)} / {len(df_all)} events for pki_type='{PKI_FILTER}'")
    else:
        df_filtered = df_all
        if PKI_FILTER:
            print(f"No 'pki_type' column found — showing all {len(df_filtered)} events")
        else:
            print(f"Showing all {len(df_filtered)} events (set PKI_FILTER to narrow)")

    columns = ['timestamp', 'event_type', 'device_fqdn', 'severity', 'pki_type', 'ca_level', 'certificate_serial']
    available_cols = [c for c in columns if c in df_filtered.columns]
    display(df_filtered[available_cols] if available_cols else df_filtered)
else:
    print("No events found. Run ./lab test to generate some.")

## Live Event Stream

Watch events in real-time. **Run the cell below and then trigger events from a terminal.**

### Triggering Events

You can trigger events in several ways. Each command sends an event to Kafka via the Mock EDR.

**Basic trigger (RSA, default):**
```bash
curl -X POST http://localhost:8082/trigger \
  -H 'Content-Type: application/json' \
  -d '{"device_id": "test-device", "scenario": "Certificate Private Key Compromise", "severity": "critical"}'
```

**ECC hierarchy:**
```bash
curl -X POST http://localhost:8082/trigger \
  -H 'Content-Type: application/json' \
  -d '{"device_id": "ecc-sensor", "scenario": "IoT Device Cloning Detected", "severity": "critical", "pki_type": "ecc"}'
```

**Post-Quantum hierarchy:**
```bash
curl -X POST http://localhost:8082/trigger \
  -H 'Content-Type: application/json' \
  -d '{"device_id": "pq-device", "scenario": "Ransomware Encryption Detected", "severity": "critical", "pki_type": "pqc"}'
```

**Using the `lab` CLI (recommended — issues a cert, triggers the event, and verifies revocation):**
```bash
./lab test --pki-type rsa --scenario "Certificate Private Key Compromise"
./lab test --pki-type ecc --scenario "IoT Device Cloning Detected"
./lab test --pki-type pqc --scenario "Ransomware Encryption Detected"
```

**Run all 26 scenarios at once:**
```bash
./lab test --pki-type rsa --all
```

In [ ]:
def stream_events(duration_seconds=60):
    """Stream events for a specified duration."""
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_SERVERS,
        auto_offset_reset='latest',
        enable_auto_commit=True,
        group_id='jupyter-viewer',
        consumer_timeout_ms=1000,
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )
    
    print(f"Listening for events for {duration_seconds} seconds...")
    print("Trigger events from Mock EDR to see them here.\n")
    print("-" * 80)
    
    start_time = time.time()
    event_count = 0
    
    while time.time() - start_time < duration_seconds:
        try:
            for message in consumer:
                event = message.value
                event_count += 1
                
                print(f"[{event.get('timestamp', 'N/A')}]")
                print(f"  Type: {event.get('event_type', 'unknown')}")
                print(f"  Device: {event.get('device_fqdn', 'unknown')}")
                print(f"  Severity: {event.get('severity', 'unknown')}")
                print(f"  Description: {event.get('description', 'N/A')[:60]}...")
                if event.get('certificate_serial'):
                    print(f"  Certificate: {event.get('certificate_serial')}")
                print("-" * 80)
                
        except StopIteration:
            pass
    
    consumer.close()
    print(f"\nFinished. Received {event_count} events.")

# Stream for 60 seconds (interrupt kernel to stop early)
stream_events(60)

## Event Statistics

The cell below reads up to 1,000 events from the topic and computes aggregate counts.

- **Events by Type** — maps to the 26 event types listed above. High counts for a single type may indicate a focused attack simulation or repeated `./lab test` runs with the same scenario.
- **Events by Severity** — `critical` and `high` events trigger immediate certificate revocation. `medium` and `low` events are logged but may not trigger revocation depending on the EDA rulebook configuration.
- **Events by Device** — shows which device FQDNs appear most frequently. In the lab, device names are generated by `./lab test` (e.g., `testdevice-<timestamp>.cert-lab.local`).

In [ ]:
# Get all events for statistics
all_events = get_recent_events(1000)

if all_events:
    df = pd.DataFrame(all_events)
    
    print(f"Total Events: {len(df)}")
    print("\nEvents by Type:")
    if 'event_type' in df.columns:
        print(df['event_type'].value_counts().to_string())
    
    print("\nEvents by Severity:")
    if 'severity' in df.columns:
        print(df['severity'].value_counts().to_string())
    
    print("\nEvents by Device:")
    if 'device_fqdn' in df.columns:
        print(df['device_fqdn'].value_counts().head(10).to_string())
else:
    print("No events to analyze.")